# Japanese demographics

In [4]:
import os

APP_ID = os.environ['APP_ID']
LANG = 'J'
BASE_URL = 'https://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?appId={id}&lang={lang}'.format(id=APP_ID,
                                                                                                    lang=LANG)

In [5]:
import requests
import pandas
from datetime import datetime

json = requests.get('{url}&statsDataId=0003411561'.format(url=BASE_URL)).json()
stats = json['GET_STATS_DATA']['STATISTICAL_DATA']
labels = stats['CLASS_INF']['CLASS_OBJ']
labelMap = {label['@id']: {l['@code']: l['@name'] for l in label['CLASS']} for label in labels}
data = stats['DATA_INF']['VALUE']
for d in data:
    d['@cat01'] = labelMap['cat01'][d['@cat01']]
    d['@time'] = datetime.strptime(labelMap['time'][d['@time']], '%Y年')
df = pandas.DataFrame(data)
df = df.rename(columns={'@cat01': 'Type', '@time': 'Year', '@unit': 'Unit', '$': 'Value'})
df['Value'] = df['Value'].apply(pandas.to_numeric, errors='coerce')
df

,Type,Year,Unit,Value
0,出生数,2021-01-01,人,811622.00
1,出生数,2020-01-01,人,840835.00
2,出生数,2019-01-01,人,865239.00
3,出生数,2018-01-01,人,918400.00
4,出生数,2017-01-01,人,946146.00
...,...,...,...,...
3235,離婚率,1903-01-01,人口千対,1.44
3236,離婚率,1902-01-01,人口千対,1.43
3237,離婚率,1901-01-01,人口千対,1.43
3238,離婚率,1900-01-01,人口千対,1.46


In [6]:
df[df['Type'] == '出生数'].drop('Type', axis=1)

,Year,Unit,Value
0,2021-01-01,人,811622.0
1,2020-01-01,人,840835.0
2,2019-01-01,人,865239.0
3,2018-01-01,人,918400.0
4,2017-01-01,人,946146.0
...,...,...,...
115,1903-01-01,人,1489816.0
116,1902-01-01,人,1510835.0
117,1901-01-01,人,1501591.0
118,1900-01-01,人,1420534.0
